In [ ]:
import cv2
from glob import glob
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras import Sequential
#from tensorflow.keras.layers import Flatten,Dense,Conv3D,MaxPool3D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [ ]:
disease_path=pathlib.Path("/content/drive/MyDrive/cnn/medical leaf")

In [ ]:
a=list(disease_path.glob("Arive-Dantu/*.jpg"))
b=list(disease_path.glob('Basale/*.jpg'))
c=list(disease_path.glob('Jackfruit/*.jpg'))

d=list(disease_path.glob('Neem/*.jpg'))
e=list(disease_path.glob('Roxburgh fig/*.jpg'))
f=list(disease_path.glob('Sinesis/*.jpg'))
g=list(disease_path.glob('jasmine/*.jpg'))
h=list(disease_path.glob('karanda/*.jpg'))
i=list(disease_path.glob('mustard/*.jpg'))
j=list(disease_path.glob('peepal/*.jpg'))

In [ ]:
len(a),len(b),len(c),len(d),len(e),len(f),len(g),len(h),len(i),len(j)

(4, 113, 56, 60, 50, 43, 71, 74, 34, 63)

In [ ]:
disease_dict={"Arive-dantu":a,
              "Basale":b,
              "Jackfruite":c,
              "Neem":d,
              "Roxburg fig":e,
              "Sinesis":f,
              "jasmine":g,
              "Karanda":h,
              "mustard":i,
              "peepal":j}
disease_class={"Arive-dantu":0,
              "Basale":1,
              "Jackfruite":2,
              "Neem":3,
              "Roxburg fig":4,
              "Sinesis":5,
              "jasmine":6,
              "Karanda":7,
              "mustard":8,
              "peepal":9}

In [ ]:
x=[]
y=[]

In [ ]:
for i in disease_dict:
  disease_name=i
  disease_path_list=disease_dict[disease_name]
  for path in disease_path_list:
    img=cv2.imread(str(path))
    img=cv2.resize(img,(224,224))
    img=img/255
    x.append(img)
    cls=disease_class[disease_name]
    y.append(cls)

In [ ]:
len(x)

568

In [ ]:
x=np.array(x)
y=np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.75,random_state=1)


In [ ]:
len(x_train),len(y_train),len(x_test),len(y_test)

(426, 426, 142, 142)

In [ ]:
x_train.shape

(426, 224, 224, 3)

In [ ]:
x_train.shape,x_test.shape


((426, 224, 224, 3), (142, 224, 224, 3))

In [ ]:
# from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

In [ ]:
# base_model = tf.keras.applications.MobileNetV2(input_shape=(32,32,3),include_top=False,weights='imagenet')

In [ ]:
# print("[INFO] summary for base model...")
# print(base_model.summary())

In [ ]:
from tensorflow.keras.layers import MaxPooling2D
from keras.layers.core import Dropout
from tensorflow.keras import datasets,layers,models
model=models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(224,224,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))




model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(10,activation='softmax'))

In [ ]:
# model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])

# # history=model.fit(xtrain,ytrain,epochs=15,validation_data=(xtest,ytest))

# #model_hist=model.fit(xtrain,ytrain,epochs=5,validation_data=(xtest,ytest),batch_size=180)



# history = model.fit(xtrain,epochs=15,
#                         validation_data=ytrain)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
history=model.fit(x_train,y_train,epochs=25,
                  validation_data=(x_test,y_test))

Epoch 1/25


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


12/14 [========================>.....] - ETA: 6s - loss: 2.3702 - accuracy: 0.2969 


KeyboardInterrupt



In [ ]:
model.save("weights.h5")

In [ ]:
from tensorflow.keras.preprocessing import image
# testing the model

def testing_image(image_directory):
  test_image=image.load_img(image_directory,target_size=(224,224))
  test_image=image.img_to_array(test_image)
  test_image=np.expand_dims(test_image,axis=0)
  test_image=test_image/255
  result=model.predict(x=test_image)
  print(result)
  if np.argmax(result)==0:
    prediction='Arive-dantu'
  elif np.argmax(result)==1:        
    prediction='Basale' 
  elif np.argmax(result)==2:
    prediction='Jackfruite'
  elif np.argmax(result)==3:
    prediction='Neem'
  elif np.argmax(result)==4:
    prediction='Roxburg fig'
  elif np.argmax(result)==5:
    prediction='Sinesis'
  elif np.argmax(result)==6:
    prediction='jasmine'
  elif np.argmax(result)==7:
    prediction="Karanda"
  elif np.argmax(result)==7:
    prediction="mustard"
  else:
    prediction="peepal"
  return prediction

In [ ]:
print(testing_image('/content/drive/MyDrive/cnn/jasmine/J-S-025.jpg'))

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
y_pred=model.predict_generator(x_test)

In [ ]:
y_pred=np.argmax(y_pred,axis=1)

In [ ]:
print("confusion matrix")

c=confusion_matrix(y_test,y_pred)
# cm=confusion_matrix(np.where(y_test),y_pred)